# Installs, Imports, Setup

In [1]:
!pip install sodapy
!pip install -U scikit-learn
# !pip install -U pandas-profiling

You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: scikit-learn in /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages (0.22)
You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# import geoencoding libraries
!pip install geopandas
!pip install geopy

You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# This will take a few minutes
# Documentation, code snippet, and token information: https://dev.socrata.com/foundry/datacatalog.cookcountyil.gov/apwk-dzx8
# This is based heavily on an NYU presentation from Nicholas Wolf and Vicky Steeves-
# Python for Harvesting Data on the Web - https://nyu-dataservices.github.io/DataHarvesting-Python/
from sodapy import Socrata
import pandas as pd
import numpy as np
from numpy import array 
from numpy import argmax 
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
import datetime
from sklearn.model_selection import train_test_split, KFold

# import pandas_profiling

from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

In [8]:
client = Socrata('datacatalog.cookcountyil.gov',
                  'UEcAAUQpAzK6Dl3x7dYTCc5cc',
                  username="kelseymarkey@gmail.com",
                  password="Idsproject1!")

init = client.get("7mck-ehwz", limit=1000000)
disp = client.get("apwk-dzx8", limit=900000)
#sent = client.get("tg8v-tm6u", limit=300000)

initiation = pd.DataFrame.from_records(init)
disposition = pd.DataFrame.from_records(disp)
#sentencing = pd.DataFrame.from_records(sent)

#should we save these 3 to csv's so we dont have to get them anew every time? -akr

In [10]:
# # Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# init.to_csv('init')
# disp.to_csv('disp')

initiation.to_csv('initiation')
disposition.to_csv('disposition')

In [11]:
# link = 'https://drive.google.com/open?id=1yzMLP0BBFGfxQZVruaF_xOxkFUmnWizk' # The shareable link
# fluff, id = link.split('=')
# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('Filename.csv')  # where is this filename coming from? why not MHI_true.csv?
# MHI_true = pd.read_csv('Filename.csv') # Dataset is now stored in a Pandas Dataframe

MHI_true = pd.read_csv('MHI_true_120419.csv')

In [13]:
MHI_true.head()

,Unnamed: 0,case_participant_id,MHI
0,186,221334022779,True
1,1356,252358604006,True
2,1601,239071986813,True
3,1918,248157954693,True
4,2027,249897124820,True


In [15]:
MHI_true.shape

(2439, 3)

In [16]:
'''
# kcm: i don't think we have to do this everytime? 

#confirm that all values of case_participant_id from sentencing are also in disposition
print(all(sentencing.case_participant_id.isin(disposition.case_participant_id)))
# test works - prints True

#confirm that all values of case_participant_id from disposition are also in initiation
print(all(disposition.case_participant_id.isin(initiation.case_participant_id)))
# test fails! prints False. this goes against our assumption. why?? what's going on here?

#are ANY of the values from disposition in initiation??
print(any(disposition.case_participant_id.isin(initiation.case_participant_id)))
# test works - there are at least some from dispo in init'''

"\n# kcm: i don't think we have to do this everytime? \n\n#confirm that all values of case_participant_id from sentencing are also in disposition\nprint(all(sentencing.case_participant_id.isin(disposition.case_participant_id)))\n# test works - prints True\n\n#confirm that all values of case_participant_id from disposition are also in initiation\nprint(all(disposition.case_participant_id.isin(initiation.case_participant_id)))\n# test fails! prints False. this goes against our assumption. why?? what's going on here?\n\n#are ANY of the values from disposition in initiation??\nprint(any(disposition.case_participant_id.isin(initiation.case_participant_id)))\n# test works - there are at least some from dispo in init"

In [18]:
initiation.head()

,case_id,case_participant_id,offense_category,primary_charge,charge_id,charge_version_id,charge_offense_title,chapter,act,section,...,incident_begin_date,arrest_date,law_enforcement_agency,received_date,arraignment_date,updated_offense_category,charge_count,incident_city,unit,incident_end_date
0,64999851671,260122253823,PROMIS Conversion,False,2037067609997,462725085037,FIRST DEGREE MURDER,720,5,9-1(a)(2),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,2,NaN,NaN,NaN
1,64999851671,260122253823,PROMIS Conversion,False,2037073820980,462726494735,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,3,NaN,NaN,NaN
2,64999851671,260122253823,PROMIS Conversion,False,2037079144680,462727770177,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,4,NaN,NaN,NaN
3,64999851671,260122253823,PROMIS Conversion,False,2037080919246,462728172948,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,5,NaN,NaN,NaN
4,64999851671,260122253823,PROMIS Conversion,False,2037084172619,462728911361,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,6,NaN,NaN,NaN


In [20]:
initiation.shape

(904719, 27)

In [21]:
disposition.shape

(821723, 31)

In [22]:
disposition.head()

,case_id,case_participant_id,offense_category,primary_charge,charge_id,charge_version_id,disposition_charged_offense_title,disposition_charged_chapter,disposition_charged_act,disposition_charged_section,...,incident_begin_date,arrest_date,law_enforcement_agency,incident_city,received_date,arraignment_date,updated_offense_category,charge_count,unit,incident_end_date
0,116240177428,908314430833,Domestic Battery,True,2077894472647,472118710537,AGGRAVATED DOMESTIC BATTERY,720,5,12-3.3(a),...,12/8/2011 12:00:00 AM,12/29/2011 12:10:00 PM,CHICAGO PD,Chicago,1/3/2012 12:00:00 AM,2/1/2012 12:00:00 AM,Domestic Battery,1,NaN,NaN
1,116240221230,908314782607,Retail Theft,True,2074969691100,471445411764,RETAIL THEFT,720,5,16A-3(a),...,1/3/2012 12:00:00 AM,1/3/2012 12:05:00 PM,CHICAGO PD,Chicago,1/3/2012 12:00:00 AM,1/31/2012 12:00:00 AM,Retail Theft,1,NaN,NaN
2,116240265032,908315368896,Narcotics,True,2074970874144,471445680278,POSSESSION OF A CONTROLLED SUBSTANCE,720,570,402(c),...,12/29/2011 12:00:00 AM,12/29/2011 11:55:00 AM,CHICAGO PD,Chicago,1/3/2012 12:00:00 AM,2/9/2012 12:00:00 AM,Narcotics,1,District 12 - Monroe,NaN
3,116240352636,908315720670,Narcotics,True,2074973831755,471446351563,[POSSESSION OF CANNABIS WITH INTENT TO DELIVER...,720,550,5(d),...,12/29/2011 12:00:00 AM,12/29/2011 2:11:00 PM,CHICAGO PD,Chicago,1/3/2012 12:00:00 AM,2/8/2012 12:00:00 AM,Narcotics,1,District 4 - South Chicago,NaN
4,116240396439,908316658734,Narcotics,True,2074979451216,487873767563,POSSESSION OF A CONTROLLED SUBSTANCE,720,570,402(c),...,12/29/2011 12:00:00 AM,12/29/2011 11:00:00 AM,CHICAGO PD,Chicago,1/3/2012 12:00:00 AM,2/8/2012 12:00:00 AM,Narcotics,1,District 11 - Harrison,NaN


# Data Prep: Filtering


In [23]:
#make copies of initiation & disposition dfs where case_participant_id is an int
#doing this in order to be compatible with MHI_true
init_int = initiation.copy()
init_int.case_participant_id = init_int.case_participant_id.astype(int)
dispo_int = disposition.copy()
dispo_int.case_participant_id = dispo_int.case_participant_id.astype(int)

In [24]:
dispo_int.case_participant_id

0          908314430833
1          908314782607
2          908315368896
3          908315720670
4          908316658734
              ...      
821718    1101083006411
821719    1101084648023
821720    1101121467024
821721    1101123108635
821722    1101125805568
Name: case_participant_id, Length: 821723, dtype: int64

In [25]:
init_int.case_participant_id

0          260122253823
1          260122253823
2          260122253823
3          260122253823
4          260122253823
              ...      
904714    1101545354565
904715    1101546878918
904716    1101548872304
904717    1101549575851
904718    1101549575851
Name: case_participant_id, Length: 904719, dtype: int64

In [26]:
dispo_int.head()

,case_id,case_participant_id,offense_category,primary_charge,charge_id,charge_version_id,disposition_charged_offense_title,disposition_charged_chapter,disposition_charged_act,disposition_charged_section,...,incident_begin_date,arrest_date,law_enforcement_agency,incident_city,received_date,arraignment_date,updated_offense_category,charge_count,unit,incident_end_date
0,116240177428,908314430833,Domestic Battery,True,2077894472647,472118710537,AGGRAVATED DOMESTIC BATTERY,720,5,12-3.3(a),...,12/8/2011 12:00:00 AM,12/29/2011 12:10:00 PM,CHICAGO PD,Chicago,1/3/2012 12:00:00 AM,2/1/2012 12:00:00 AM,Domestic Battery,1,NaN,NaN
1,116240221230,908314782607,Retail Theft,True,2074969691100,471445411764,RETAIL THEFT,720,5,16A-3(a),...,1/3/2012 12:00:00 AM,1/3/2012 12:05:00 PM,CHICAGO PD,Chicago,1/3/2012 12:00:00 AM,1/31/2012 12:00:00 AM,Retail Theft,1,NaN,NaN
2,116240265032,908315368896,Narcotics,True,2074970874144,471445680278,POSSESSION OF A CONTROLLED SUBSTANCE,720,570,402(c),...,12/29/2011 12:00:00 AM,12/29/2011 11:55:00 AM,CHICAGO PD,Chicago,1/3/2012 12:00:00 AM,2/9/2012 12:00:00 AM,Narcotics,1,District 12 - Monroe,NaN
3,116240352636,908315720670,Narcotics,True,2074973831755,471446351563,[POSSESSION OF CANNABIS WITH INTENT TO DELIVER...,720,550,5(d),...,12/29/2011 12:00:00 AM,12/29/2011 2:11:00 PM,CHICAGO PD,Chicago,1/3/2012 12:00:00 AM,2/8/2012 12:00:00 AM,Narcotics,1,District 4 - South Chicago,NaN
4,116240396439,908316658734,Narcotics,True,2074979451216,487873767563,POSSESSION OF A CONTROLLED SUBSTANCE,720,570,402(c),...,12/29/2011 12:00:00 AM,12/29/2011 11:00:00 AM,CHICAGO PD,Chicago,1/3/2012 12:00:00 AM,2/8/2012 12:00:00 AM,Narcotics,1,District 11 - Harrison,NaN


In [27]:
dispo_int.shape

(821723, 31)

In [28]:
init_int.head()

,case_id,case_participant_id,offense_category,primary_charge,charge_id,charge_version_id,charge_offense_title,chapter,act,section,...,incident_begin_date,arrest_date,law_enforcement_agency,received_date,arraignment_date,updated_offense_category,charge_count,incident_city,unit,incident_end_date
0,64999851671,260122253823,PROMIS Conversion,False,2037067609997,462725085037,FIRST DEGREE MURDER,720,5,9-1(a)(2),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,2,NaN,NaN,NaN
1,64999851671,260122253823,PROMIS Conversion,False,2037073820980,462726494735,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,3,NaN,NaN,NaN
2,64999851671,260122253823,PROMIS Conversion,False,2037079144680,462727770177,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,4,NaN,NaN,NaN
3,64999851671,260122253823,PROMIS Conversion,False,2037080919246,462728172948,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,5,NaN,NaN,NaN
4,64999851671,260122253823,PROMIS Conversion,False,2037084172619,462728911361,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,6,NaN,NaN,NaN


In [29]:
init_int.shape

(904719, 27)

In [30]:
#create filtered version of init_int that includes only the cp ids in both dispo & init

#save cp id values as lists
init_ids = list(init_int.case_participant_id.values)
dispo_ids = list(dispo_int.case_participant_id.values)

#create list of unique cp ids that are in both init & dispo
to_keep = list(set(init_ids).intersection(set(dispo_ids)))

#create filtered df with only records from the intersection of cp ids
init_filtered = init_int[init_int['case_participant_id'].isin(to_keep)]

# confirm that all cp ids in init_filtered are in initiation
# print(all(init_filtered.case_participant_id.isin(initiation.case_participant_id)))
# confirmed - prints True

# confirm that all cp ids in init_filtered are in disposition
# print(all(init_filtered.case_participant_id.isin(disposition.case_participant_id)))
# confirmed - prints True

# trying to save as csv
init_filtered.to_csv('init_filtered.csv')


In [35]:
# print(init_ids, dispo_ids, to_keep, init_filtered.head())
init_ids[0:10]

[260122253823,
 260122253823,
 260122253823,
 260122253823,
 260122253823,
 260122253823,
 260122253823,
 260122253823,
 260122253823,
 260122253823]

In [36]:
dispo_ids[0:10]

[908314430833,
 908314782607,
 908315368896,
 908315720670,
 908316658734,
 908317010508,
 908317479540,
 908317479540,
 908317479540,
 908317479540]

In [38]:
to_keep[0:10]

[929789640705,
 977562763265,
 900115988484,
 965988581382,
 975038316551,
 927265193992,
 984088051720,
 1022811439110,
 1059010379787,
 924740747278]

In [40]:
init_filtered.head()

,case_id,case_participant_id,offense_category,primary_charge,charge_id,charge_version_id,charge_offense_title,chapter,act,section,...,incident_begin_date,arrest_date,law_enforcement_agency,received_date,arraignment_date,updated_offense_category,charge_count,incident_city,unit,incident_end_date
0,64999851671,260122253823,PROMIS Conversion,False,2037067609997,462725085037,FIRST DEGREE MURDER,720,5,9-1(a)(2),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,2,NaN,NaN,NaN
1,64999851671,260122253823,PROMIS Conversion,False,2037073820980,462726494735,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,3,NaN,NaN,NaN
2,64999851671,260122253823,PROMIS Conversion,False,2037079144680,462727770177,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,4,NaN,NaN,NaN
3,64999851671,260122253823,PROMIS Conversion,False,2037080919246,462728172948,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,5,NaN,NaN,NaN
4,64999851671,260122253823,PROMIS Conversion,False,2037084172619,462728911361,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,3/12/2007 12:00:00 AM,5/22/2011 6:51:00 PM,PROMIS Data Conversion,5/24/2011 12:00:00 AM,7/11/2011 12:00:00 AM,Homicide,6,NaN,NaN,NaN


In [41]:
init_filtered.shape

(743804, 27)

In [42]:
#CHECKPOINT
# need to run the prior if we don't have 'init_filtered.csv' - amber notes
#start from here if don't want to rerun everything before?
#move to new notebook?
init_filtered = pd.read_csv('init_filtered.csv', index_col=0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data Prep: Cleaning, converting, dealing with null values, creating columns for missing age

In [43]:
#create new df to work on for processing/cleaning
init_clean = init_filtered.copy()

In [44]:
#process/clean data:
#convert age to int

#new df to work on
init_age_int = init_clean.copy()

#look at unique values of age
# print(init_age_int['age_at_incident'].unique())


#turn nan to '0' so can process into ints
init_age_int.loc[init_age_int['age_at_incident'].isnull(), 'age_at_incident'] = '0'

#change type to int
init_age_int.age_at_incident = init_age_int.age_at_incident.astype(int)

#look at unique values again
# print(init_age_int['age_at_incident'].unique())

#how many are messed up? all fixed in cell below -kcm
#print('Number of records with null age: ', len(init_age_int[init_age_int['age_at_incident']==0]))
# print('Number of records with age > 100: ', len(init_age_int[init_age_int['age_at_incident']>100]))

init_clean = init_age_int


In [45]:
#process/clean data:
#creating 2 new features: age > 100, age_ unknown

#new df to work on
init_age = init_clean.copy()

#creating new binary colmns for age over 100 and age unknown (1 = true)
init_age['age_over_100'] = (init_age.age_at_incident > 100).astype(int) 
init_age['age_unknown'] = (init_age.age_at_incident == 0).astype(int)

#replacing > 100 and 0 values with nan, calculating median, then changing nan to mean
init_age.age_at_incident.replace(0, np.NaN, inplace=True)
a = np.array(init_age['age_at_incident'].values.tolist())
init_age['age_at_incident'] = np.where(a > 100, np.nan, a).tolist()
# init_age.age_at_incident.isna().sum() # 10562, count correct
median = init_age['age_at_incident'].median() # median 29, mean 31
init_age.age_at_incident.replace(np.NaN, median, inplace=True) 
# final median still 29, mean 32 

init_clean = init_age

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  


In [46]:
#process/clean data:
#convert time features to datetime objects

#create new df to work on
init_datetime = init_clean.copy()

#list of time features
time_features = ['event_date', 'incident_begin_date', 'arrest_date', 
                 'received_date', 'arraignment_date', 'incident_end_date']

for col in time_features:
  #init_datetime[col] = pd.to_datetime(init_datetime[col], format='%m/%d/%Y %I:%M:%S %p')
  #this version gets the following error: 'OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 2912-01-18 00:00:00'

  #turn nan to '01/01/1900 12:00:00 AM' so can process them as strings
  init_datetime.loc[init_datetime[col].isnull(), col] = '01/01/1900 12:00:00 AM'

  #make sure everything is strings
  init_datetime[col] = init_datetime[col].astype(str)
  
  #change type to datetime
  init_datetime[col] = init_datetime[col].map(lambda x: pd.datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))

# amber question - why do we want to do init_clean = init_datetime when we already saved our changes to init_datetime? 
# akr answer - i did this with each of the cleaning methods -  made a copy and then saved it back when i was done. i just thought it was safer?
init_clean = init_datetime

In [47]:
#process/clean data:
#clean up gender 

#create new df to work in
init_gender = init_clean.copy()

#look at unique values
print(init_gender['gender'].unique())

#how many are messed up?
print('Number of records with gender=Male name, no gender given: ', len(init_gender[init_gender['gender']=='Male name, no gender given']))
print('Number of records with gender=Unknown: ', len(init_gender[init_gender['gender']=='Unknown']))
print('Number of records with gender=Unknown Gender: ', len(init_gender[init_gender['gender']=='Unknown Gender']))
print('Number of records with null Gender: ', len(init_gender[init_gender['gender'].isnull()]))

#turning all except 'Male' and 'Female' to 'Unknown'
init_gender.loc[init_gender['gender'].isnull(), 'gender'] = 'Unknown'
init_gender.loc[init_gender['gender']=='Male name, no gender given', 'gender'] = 'Unknown'
init_gender.loc[init_gender['gender']=='Unknown Gender', 'gender'] = 'Unknown'

init_clean = init_gender

['Male' 'Female' nan 'Male name, no gender given' 'Unknown Gender'
 'Unknown']
Number of records with gender=Male name, no gender given:  4
Number of records with gender=Unknown:  7
Number of records with gender=Unknown Gender:  2
Number of records with null Gender:  2722


In [48]:
init_clean.head()
# amber question - is this similar to OHE? isn't this same process of cleaning up gender the same as OHE?
#akr answer - no it's totally different. i combined 3 unknown categories into a single unknown category. didn't create dummy variables here at all.

,case_id,case_participant_id,offense_category,primary_charge,charge_id,charge_version_id,charge_offense_title,chapter,act,section,...,law_enforcement_agency,received_date,arraignment_date,updated_offense_category,charge_count,incident_city,unit,incident_end_date,age_over_100,age_unknown
0,64999851671,260122253823,PROMIS Conversion,False,2037067609997,462725085037,FIRST DEGREE MURDER,720,5,9-1(a)(2),...,PROMIS Data Conversion,2011-05-24,2011-07-11 00:00:00,Homicide,2,NaN,NaN,1900-01-01,0,0
1,64999851671,260122253823,PROMIS Conversion,False,2037073820980,462726494735,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,PROMIS Data Conversion,2011-05-24,2011-07-11 00:00:00,Homicide,3,NaN,NaN,1900-01-01,0,0
2,64999851671,260122253823,PROMIS Conversion,False,2037079144680,462727770177,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,PROMIS Data Conversion,2011-05-24,2011-07-11 00:00:00,Homicide,4,NaN,NaN,1900-01-01,0,0
3,64999851671,260122253823,PROMIS Conversion,False,2037080919246,462728172948,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,PROMIS Data Conversion,2011-05-24,2011-07-11 00:00:00,Homicide,5,NaN,NaN,1900-01-01,0,0
4,64999851671,260122253823,PROMIS Conversion,False,2037084172619,462728911361,FIRST DEGREE MURDER,720,5,9-1(a)(3),...,PROMIS Data Conversion,2011-05-24,2011-07-11 00:00:00,Homicide,6,NaN,NaN,1900-01-01,0,0


In [49]:
#process/clean data:
#clean up race

#create new df to work in
init_race = init_clean.copy()

#look at unique values
print(init_race['race'].unique())

#change all to lower, to combine ASIAN and Asian
init_race['race']=init_race['race'].str.lower()

#look at unique values again
print(init_race['race'].unique())

#how many are messed up?
print(init_race['race'].value_counts())
print('Number of records with null race: ', len(init_race[init_race['race'].isnull()]))

#change nan to "unknown"
init_race['race'].fillna("unknown", inplace = True)

# could work on treatment of hispanic/latino
# leaving these values untouched for now
# existing codes speak to racial perception, if not ethnicity

init_clean = init_race

# amber question - we can use openrefine for data cleaning so it's not super manual if you guys want
# idk what that is but it sounds lit -akr

['Black' 'HISPANIC' 'White [Hispanic or Latino]'
 'White/Black [Hispanic or Latino]' 'White' nan 'Unknown' 'Asian'
 'Biracial' 'American Indian' 'Albino' 'ASIAN']
['black' 'hispanic' 'white [hispanic or latino]'
 'white/black [hispanic or latino]' 'white' nan 'unknown' 'asian'
 'biracial' 'american indian' 'albino']
black                               494039
white [hispanic or latino]          128261
white                                97785
hispanic                              9259
asian                                 5017
white/black [hispanic or latino]      4011
unknown                               1119
american indian                        385
biracial                               102
albino                                   1
Name: race, dtype: int64
Number of records with null race:  3825


In [50]:
#process/clean data:
#convert charge_count to int

#new df to work on
init_charge_count = init_clean.copy()

#look at unique values of charge_count
print(init_charge_count['charge_count'].unique())

#how many are super high??
# print(init_charge_count['charge_count'].value_counts()[-50:])

#change type to int
init_charge_count.charge_count = init_charge_count.charge_count.astype(int)

#look at highest
#print(init_charge_count.loc[init_charge_count['charge_count']>268, 'case_participant_id'].unique())

#look at 5 highest
# top5 = init_charge_count.loc[init_charge_count['charge_count']==188, 'case_participant_id'].unique()
# for person in top5:
#   print(len(init_charge_count[init_charge_count['case_participant_id']==person]))
#result is that the charge count numbers dont map perfectly to the number of rows
#this needs to be looked into more!

init_clean = init_charge_count

[  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105   1 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 24

In [51]:
#process/clean data:
#change all string features to lowercase

#new df to work on
init_lower = init_clean.copy()

string_cols = ['chapter', 'act', 'section', 'class', 'aoic', 'event', 'gender',
               'race', 'law_enforcement_agency', 'updated_offense_category', 
               'charge_offense_title', 'incident_city', 'unit']

for col in string_cols:
    init_lower[col]=init_lower[col].str.lower()
  #note that str.lower() automatically ignores null values

init_clean = init_lower

In [52]:
#process/clean data:
#deal with remaining null values

#new df to work on
init_na = init_clean.copy()

#where are there still null values?
print(init_na.columns[init_clean.isna().any()].tolist())

#replace remaining null values with "Unknown"
#chose "Unknown", because this seems to be what Cook County uses
init_na.fillna("unknown", inplace = True)

init_clean = init_na

['chapter', 'act', 'section', 'aoic', 'event', 'law_enforcement_agency', 'incident_city', 'unit']


In [0]:
#process/clean other columns?

In [53]:
#CHECKPOINT:
#save cleaned dataframe to csv

init_clean.to_csv('init_clean.csv')


In [54]:
init_clean = pd.read_csv('init_clean.csv', index_col=0)

In [55]:
init_clean.head()

,case_id,case_participant_id,offense_category,primary_charge,charge_id,charge_version_id,charge_offense_title,chapter,act,section,...,law_enforcement_agency,received_date,arraignment_date,updated_offense_category,charge_count,incident_city,unit,incident_end_date,age_over_100,age_unknown
0,64999851671,260122253823,PROMIS Conversion,False,2037067609997,462725085037,first degree murder,unknown,unknown,9-1(a)(2),...,promis data conversion,2011-05-24 00:00:00,2011-07-11 00:00:00,homicide,2,unknown,unknown,1900-01-01 00:00:00,0,0
1,64999851671,260122253823,PROMIS Conversion,False,2037073820980,462726494735,first degree murder,unknown,unknown,9-1(a)(3),...,promis data conversion,2011-05-24 00:00:00,2011-07-11 00:00:00,homicide,3,unknown,unknown,1900-01-01 00:00:00,0,0
2,64999851671,260122253823,PROMIS Conversion,False,2037079144680,462727770177,first degree murder,unknown,unknown,9-1(a)(3),...,promis data conversion,2011-05-24 00:00:00,2011-07-11 00:00:00,homicide,4,unknown,unknown,1900-01-01 00:00:00,0,0
3,64999851671,260122253823,PROMIS Conversion,False,2037080919246,462728172948,first degree murder,unknown,unknown,9-1(a)(3),...,promis data conversion,2011-05-24 00:00:00,2011-07-11 00:00:00,homicide,5,unknown,unknown,1900-01-01 00:00:00,0,0
4,64999851671,260122253823,PROMIS Conversion,False,2037084172619,462728911361,first degree murder,unknown,unknown,9-1(a)(3),...,promis data conversion,2011-05-24 00:00:00,2011-07-11 00:00:00,homicide,6,unknown,unknown,1900-01-01 00:00:00,0,0


In [56]:
init_clean.shape

(743804, 29)

# Data Prep: Get Dummies, Group By, Aggregate, Join in columns for feature engineering

In [57]:
'''
FEATURE ENGINEERING:


402 columns: if an entry in section contains 402, then 402 column value will be 1
else, it will be 0. 

applying this to init_clean before it's transformed to total_df. (we should move this up)

'''
# initialize empty column 402 
init_clean["402"] = 0


def sectionconverter(x): 
  if "402" in x: 
    new_row = 1
  else: 
    new_row = 0
  return new_row

init_clean['402'] = init_clean['section'].apply(sectionconverter)

In [58]:
# check df before OHE
init_clean.shape

(743804, 30)

In [59]:
'''
akr version of OHE / Get Dummies 
on init_clean


is this different for the OHE function I made? - amber
akr response: 
This is just a one-line call to get_dummies.
I couldn't figure out what your function was doing. 
Your version is preserved in the Old Master file if we want to go back to it?

I'm taking out some of these that require feature engineering, so they're not deleted ('incident_city', 'unit'). -Karm
akr response:
But then we wont get dummies for them so we can't use them as features.
And will we even be able to aggregate these columns if they're still strings? 
You can always look up the OG columns in init_clean for feature engineering.
I think you basically did that by adding incident_city and unit onto the section
where we add the "unsquishable" features onto the squished df, to create 
total_df. 

'''

init_clean_copy = init_clean.copy()

# specify categorical columns to get dummies for
# including all categorical variables (as a starting point)
# excluding ID numbers, dates, age
cat_cols = ['primary_charge', 'chapter', 'act', 'section', 'class', 'aoic',
       'event', 'gender', 'race', 'law_enforcement_agency', 'updated_offense_category',
       'charge_offense_title', 'incident_city', 'unit']
dummy_init = pd.get_dummies(init_clean_copy, columns=cat_cols)

In [60]:
dummy_init.shape

# trying to save to csv
# dummy_init.to_csv('dummy_init.csv')
# when akr tried, took foreverrrrrr. eventually gave up
# when at tried, file was too big, ran out of ram 

(743804, 5205)

In [61]:
#now attempting groupby

#sometimes crashes bc we run out of ram
init_grouped = dummy_init.groupby('case_participant_id')
#behaving well now, not sure what changed

In [62]:
#now aggregate (aka squish down to 1 row)
#we want certain things to be summed, certain things to be averaged

#anything that might be different between charges --> sum
#if all same across charges --> avg
#charge count--> want max
#if max(charge_count) != number of rows, then we need to rethink this...
#something about # of primary_charge = False ?

#do we want to preserve any info about the top charge?? maybe later in fancier models

In [63]:
#checking which features are the same for every unique case_participant_id
clean_groups = init_clean.groupby('case_participant_id')
same_within_group = []
diff_within_group = []
for col in init_clean.columns:
  if any(clean_groups[col].nunique()>1):
    diff_within_group.append(col)
  else:
    same_within_group.append(col)

print('same: ', same_within_group)
print('different: ', diff_within_group)


# I think we should remove the variables that are the same from OHE to reduce 
# the size of the df at this point. Once we downsample, we'll have an unhealthy
# observation to column ratio. It's only meaningful to isolate a specific 
# subcategory if we think a subcategory is more meaningful than others, right? 
# or if we want to check interactions? It's not necessary to have a binary for 
# each subcategory, only some. We should talk about this before modeling. -Karm

# akr response: 
# i think this is def a good idea. 
# dont think it applies in this section tho. 
# not sure what you mean by sucategories? 
# let's def talk about this!

same:  ['case_id', 'case_participant_id', 'offense_category', 'event', 'event_date', 'age_at_incident', 'gender', 'race', 'incident_begin_date', 'arrest_date', 'law_enforcement_agency', 'received_date', 'arraignment_date', 'updated_offense_category', 'incident_city', 'unit', 'incident_end_date', 'age_over_100', 'age_unknown']
different:  ['primary_charge', 'charge_id', 'charge_version_id', 'charge_offense_title', 'chapter', 'act', 'section', 'class', 'aoic', 'charge_count', '402']


In [64]:
# create dictionary where keys are columns in initiation
# values are what we will do with them when we aggregrate the groupby object (ie, squish down to 1 row)

#we take the median for anything that's always the same within unique case_participant_id groups
#note that every individual value should be equal to the median
#excluding datetime objects (and incident_city and unit) because i think you can only aggregate numbers
#exclusing MHI because it is True/False
to_median = ['age_at_incident', 'age_over_100', 'age_unknown', 'gender', 'race', 'law_enforcement_agency', 'updated_offense_category']
agg_dict_short = {x : 'median' for x in to_median}

#we take the highest charge_count to represent the total number of charges
agg_dict_short['charge_count'] = 'max'

#we take the highest "402" (binary - 0/1) to capture the 1s 
agg_dict_short['402'] = 'max'

#list of categorical features that vary within unique case_participant_id groups
to_sum = ['chapter', 'act', 'section', 'class', 'aoic', 'charge_offense_title']
#we will sum these features when we aggregrate
for item in to_sum:
  agg_dict_short[item] = 'sum'

print(agg_dict_short)

{'age_at_incident': 'median', 'age_over_100': 'median', 'age_unknown': 'median', 'gender': 'median', 'race': 'median', 'law_enforcement_agency': 'median', 'updated_offense_category': 'median', 'charge_count': 'max', '402': 'max', 'chapter': 'sum', 'act': 'sum', 'section': 'sum', 'class': 'sum', 'aoic': 'sum', 'charge_offense_title': 'sum'}


In [65]:
#now use agg_dict_short to create dictionary of the columns in dummy_init
#because dummy_init is what we are really doing groupby/aggregate on
agg_dict_long = {}

#loop through every key, val pair in agg_dict_short
for key, val in agg_dict_short.items():

  #check if key is not in the list of cat_cols that got turned into dummy variables
  if key not in cat_cols:

    #add key, val pair to agg_dict_long
    agg_dict_long[key] = val

  else:

    #loop through every column in dummy_init
    for col in dummy_init:

      #check if the key is a substring of the dummy_init column name
      if key+'_' in col:

        #add val to agg_dict_long with the dummy_init column name as new key
        agg_dict_long[col] = val

# print(agg_dict_long)

In [66]:
#now the actual aggregate call on the groupby object
init_squish = init_grouped.aggregate(agg_dict_long)

# note index is now cp id's
# do we need to worry about ints vs floats?

In [67]:
init_squish.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,charge_offense_title_violation of bail bond,charge_offense_title_violation of the murderer and violent offender against youth registration act,charge_offense_title_violation of the sex offender registration act,charge_offense_title_violation of the stalking no contact order act,charge_offense_title_wic fraud/possess authz/>$10k,charge_offense_title_window peeping 3rd+,charge_offense_title_wire fraud,charge_offense_title_written prescription req/1st,charge_offense_title_written prescription req/subq,charge_offense_title_wrker comp/fraudulent claim
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,22.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
272161011760,29.0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
864286527653,29.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
882206007016,34.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
882242005211,41.0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
init_squish.shape

(289094, 4821)

In [69]:
#append MHI column to init_squish

#initialize MHI column to false
init_squish['MHI'] = 0

#change MHI to 1 for the cp ids which appear in MHI_true
init_squish.loc[init_squish.index.isin(MHI_true.case_participant_id.values), 'MHI']=1

#testing that the MHI appending worked as expected
#print(init_squish.loc[678961772028])
#print(init_squish.head())
#appears to have worked

#do we need to worry about ints vs floats?

In [70]:
print('number of positive cases in squished df: ', len(init_squish[init_squish.MHI==1]))

number of positive cases in squished df:  2212


In [71]:
# trying to save to csv
# init_squish.to_csv('init_squish.csv')
# too big, not working

In [72]:
# HERE IS WHERE WE ARE MAKING TOTAL DF 

In [73]:
'''
Adding Unsquishable Features to "squished" DF

Note that this is where we are creating total_Df
'''

#list of time features
time_features = ['event_date', 'incident_begin_date', 'arrest_date', 
                 'received_date', 'arraignment_date', 'incident_end_date']

#making df copies to work on
total_df = init_squish.copy()
time_df = init_clean.copy()

#make list of columns to keep/delete from init_clean
to_delete = list(time_df.columns)
to_keep = time_features + ['case_participant_id', 'primary_charge', 'incident_city', 'unit']  # are there any other variables we want to keep?
to_delete  = list(set(to_delete) - set(to_keep))

#drop the to_delete list, keep only the time features
time_df = time_df.drop(columns=to_delete)

#probably unecessary/superstitious, but im sorting by primary charge
time_df.sort_values(by=['primary_charge'], ascending=False, inplace = True) 

#don't need primary charge anymore; dropping it
time_df.drop(columns=['primary_charge'], inplace=True)

#drop all duplicate rows, ie all non-primary charge rows
time_df.drop_duplicates(inplace=True)

#set index of time_df
time_df.set_index('case_participant_id', inplace=True)

#confirm that time_df has as many rows as init_squish now
#print(len(init_squish)==len(time_df))
#test works, prints True

#now add our time columns on to total_df
total_df = total_df.join(time_df, on='case_participant_id')

print(total_df.shape)

(289094, 4830)


In [75]:
total_df.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,charge_offense_title_wrker comp/fraudulent claim,MHI,event_date,incident_begin_date,arrest_date,received_date,arraignment_date,incident_city,unit,incident_end_date
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,22.0,0,0,0,1,0,0,0,0,0,...,0,0,2011-06-20 00:00:00,2007-03-12 00:00:00,2011-05-22 18:51:00,2011-05-24 00:00:00,2011-07-11 00:00:00,unknown,unknown,1900-01-01 00:00:00
272161011760,29.0,0,1,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,2009-06-25 00:00:00,2009-07-14 14:34:00,2012-01-27 00:00:00,1900-01-01 00:00:00,unknown,unknown,1900-01-01 00:00:00
864286527653,29.0,0,0,0,1,0,0,0,0,0,...,0,0,2010-04-28 00:00:00,2010-04-07 00:00:00,2010-04-07 20:44:00,2011-01-31 00:00:00,2010-05-19 00:00:00,unknown,unknown,1900-01-01 00:00:00
882206007016,34.0,0,0,0,1,0,0,0,0,0,...,0,0,2011-01-05 00:00:00,2010-12-15 00:00:00,2010-12-15 20:40:00,2011-01-31 00:00:00,2011-01-31 00:00:00,chicago,unknown,1900-01-01 00:00:00
882242005211,41.0,0,0,0,1,0,0,0,0,0,...,0,0,2011-08-05 00:00:00,2011-07-17 00:00:00,2011-07-17 19:05:00,2011-07-17 00:00:00,1900-01-01 00:00:00,chicago,unknown,1900-01-01 00:00:00


In [76]:
total_df.columns

Index(['age_at_incident', 'age_over_100', 'age_unknown', 'gender_female',
       'gender_male', 'gender_unknown', 'race_albino', 'race_american indian',
       'race_asian', 'race_biracial',
       ...
       'charge_offense_title_wrker comp/fraudulent claim', 'MHI', 'event_date',
       'incident_begin_date', 'arrest_date', 'received_date',
       'arraignment_date', 'incident_city', 'unit', 'incident_end_date'],
      dtype='object', length=4830)

# Feature Engineering

In [77]:
''' 
FEATURE ENGINEERING:

kcm weekday feature

creating binary weekday column: 0 = Sat/Sun, 1 Otherwise'''

arrest_date_dt = pd.to_datetime(total_df.arrest_date) #converting to datetime
arrest_date_dt = arrest_date_dt.replace(datetime.datetime(1900, 1, 1, 0, 00), np.nan) #replacing 1900 with NaT

total_df['weekday'] = arrest_date_dt #instantiating new column
total_df['weekday'] = np.where(total_df.weekday.isnull(), np.nan, total_df.weekday.dt.dayofweek < 5)
# converting NaT to np.nan, checking if weekday ^

#total_df['weekday'].loc[883102444154] #here is a nan for testing
#how should this handle NaN values of 1900? -akr .. fixed, now 1900 dates are NaN -kcm
# QUESTION: these are floats, do they need to be ints? not sure how to do this with nans

total_df.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,MHI,event_date,incident_begin_date,arrest_date,received_date,arraignment_date,incident_city,unit,incident_end_date,weekday
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,22.0,0,0,0,1,0,0,0,0,0,...,0,2011-06-20 00:00:00,2007-03-12 00:00:00,2011-05-22 18:51:00,2011-05-24 00:00:00,2011-07-11 00:00:00,unknown,unknown,1900-01-01 00:00:00,0.0
272161011760,29.0,0,1,0,1,0,0,0,0,0,...,0,1900-01-01 00:00:00,2009-06-25 00:00:00,2009-07-14 14:34:00,2012-01-27 00:00:00,1900-01-01 00:00:00,unknown,unknown,1900-01-01 00:00:00,1.0
864286527653,29.0,0,0,0,1,0,0,0,0,0,...,0,2010-04-28 00:00:00,2010-04-07 00:00:00,2010-04-07 20:44:00,2011-01-31 00:00:00,2010-05-19 00:00:00,unknown,unknown,1900-01-01 00:00:00,1.0
882206007016,34.0,0,0,0,1,0,0,0,0,0,...,0,2011-01-05 00:00:00,2010-12-15 00:00:00,2010-12-15 20:40:00,2011-01-31 00:00:00,2011-01-31 00:00:00,chicago,unknown,1900-01-01 00:00:00,1.0
882242005211,41.0,0,0,0,1,0,0,0,0,0,...,0,2011-08-05 00:00:00,2011-07-17 00:00:00,2011-07-17 19:05:00,2011-07-17 00:00:00,1900-01-01 00:00:00,chicago,unknown,1900-01-01 00:00:00,0.0


In [78]:
'''
FEATURE ENGINEERING:
season 

'''
from datetime import date, datetime

arrest_date_dt = pd.to_datetime(total_df.arrest_date) #converting to datetime
total_df['day_of_year'] = arrest_date_dt.dt.dayofyear # making day of year column- ordinal date

#checking for various seasons
total_df['season'] = np.where(total_df['day_of_year'] <= 80, 'winter', total_df['day_of_year'])
total_df['season'] = np.where((173 > total_df['day_of_year']) &  (total_df['day_of_year'] > 80), 'spring', total_df['season'])
total_df['season'] = np.where((267 > total_df['day_of_year']) &  (total_df['day_of_year'] >= 173), 'summer', total_df['season'])
total_df['season'] = np.where((356 > total_df['day_of_year']) &  (total_df['day_of_year'] >= 267), 'fall', total_df['season'])
total_df['season'] = np.where(356 < total_df['day_of_year'], 'winter', total_df['season'])

total_df = total_df.drop(columns = 'day_of_year') # dropping day of year column  
total_df = pd.get_dummies(total_df, columns = ['season'])
total_df.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,arraignment_date,incident_city,unit,incident_end_date,weekday,season_356,season_fall,season_spring,season_summer,season_winter
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,22.0,0,0,0,1,0,0,0,0,0,...,2011-07-11 00:00:00,unknown,unknown,1900-01-01 00:00:00,0.0,0,0,1,0,0
272161011760,29.0,0,1,0,1,0,0,0,0,0,...,1900-01-01 00:00:00,unknown,unknown,1900-01-01 00:00:00,1.0,0,0,0,1,0
864286527653,29.0,0,0,0,1,0,0,0,0,0,...,2010-05-19 00:00:00,unknown,unknown,1900-01-01 00:00:00,1.0,0,0,1,0,0
882206007016,34.0,0,0,0,1,0,0,0,0,0,...,2011-01-31 00:00:00,chicago,unknown,1900-01-01 00:00:00,1.0,0,1,0,0,0
882242005211,41.0,0,0,0,1,0,0,0,0,0,...,1900-01-01 00:00:00,chicago,unknown,1900-01-01 00:00:00,0.0,0,0,0,1,0


In [79]:
'''
FEATURE ENGINEERING:
incident length

akr notes for kcm: 
idk how to catch the null values in start/end. (so we can fill in 0 for incident_length)
might be easier to change the way we're converting nulls to datetime back in the data prep section
my 1900 thing is pretty stupid, i just didnt know how else to do it
'''

# convert begin and end dates to datetime
begin_date_dt = pd.to_datetime(total_df.incident_begin_date)
begin_date_dt = begin_date_dt.replace(datetime(1900, 1, 1, 0, 00), np.nan) #replacing 1900 with NaT
end_date_dt = pd.to_datetime(total_df.incident_end_date)
end_date_dt = end_date_dt.replace(datetime(1900, 1, 1, 0, 00), np.nan) #replacing 1900 with NaT

# create incident_length column by subtracting, this will be NaT if either is NaT
total_df['incident_length'] = (end_date_dt - begin_date_dt)

# This should replace NaT with 0 
total_df['incident_length'] = np.where(total_df.incident_length.isnull(), 0, total_df.incident_length)

# Convert all timestamps to days 
total_df['incident_length'] = total_df.incident_length.dt.days

# delete everything after the space 
# assumes that the string is formatted '_____ days ...'
# might be a better way to do this part using datetime.days, but i couldnt get it to work
#total_df['incident_length'] = total_df['incident_length'].map(lambda x: x.partition(' ')[0])
#total_df['incident_length'] = total_df['incident_length'].astype(int)

#Testing
# Has end date but not begin date: 184594, 58125, 223173
# end_date_dt.iloc[58125] - begin_date_dt.iloc[58125] #testing- has end_date but not begin
# Has begin date but not end: 1256, 920, many more.. 

In [80]:
print(type(total_df.incident_length.iloc[245]))
total_df.incident_length.value_counts()

<class 'numpy.int64'>


0       279416
1         1998
2          394
3          245
4          197
         ...  
2131         1
338          1
3411         1
1618         1
8188         1
Name: incident_length, Length: 1123, dtype: int64

In [ ]:
# THIS DOES NOT RUN

In [81]:
#feature engineering:
#geoencode incident_city

# columns related to location: incident_city, unit
uniqueCities = total_df['incident_city'].unique() # 264 ? cities

# add state to city names in unique list
uniquePlaces =  [(i + ", Illinois") for i in uniqueCities]
# drop Unknown -- don't "locate" cities indicated as unknown
uniquePlaces = uniquePlaces[1:]
# convert to df
uniquePlaces_df = pd.DataFrame(uniquePlaces)

# geoencode unique list
# 1 - conveneint function to delay between geocoding calls
locator = Nominatim(user_agent='myGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2 - create location column
uniquePlaces_df['location'] = uniquePlaces_df[0].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
uniquePlaces_df['point'] = uniquePlaces_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split tuple into longitude and latitude (and altitude)
uniquePlaces_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(uniquePlaces_df['point'].tolist(), index=uniquePlaces_df.index)

# remove state from city names
uniquePlaces_df[0]=uniquePlaces_df[0].str[:-10]

# copy df
df=total_df.copy()

# set default values (will remain 0 if incidenct_city is unknown)
df['latitude'] = 0
df['longitude'] = 0

# assign appropriate long/lat
for i in range(len(uniquePlaces_df)):
    samePlace = uniquePlaces_df[0][i]
    lat = uniquePlaces_df['latitude'][i]
    lon = uniquePlaces_df['longitude'][i]

# update lat and lon if df['incident_city']==uniquePlaces_df['0']
df['latitude'] = np.where((df.incident_city == samePlace), lat, df.latitude)
df['longitude'] = np.where((df.incident_city == samePlace), lon, df.longitude)

# save clean data
total_df = df

RateLimiter caught an error, retrying (0/2 tries). Called with (*('chicago, Illinois',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Library/Frameworks/Pyt

RateLimiter caught an error, retrying (0/2 tries). Called with (*('harvey, Illinois',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Library/Frameworks/Pyth

RateLimiter caught an error, retrying (0/2 tries). Called with (*('hanover park, Illinois',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Library/Framework

RateLimiter caught an error, retrying (0/2 tries). Called with (*('schaumburg, Illinois',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Library/Frameworks/

RateLimiter caught an error, retrying (0/2 tries). Called with (*('bellwood, Illinois',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Library/Frameworks/Py

RateLimiter caught an error, retrying (0/2 tries). Called with (*('wilmette, Illinois',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Library/Frameworks/Py

RateLimiter caught an error, retrying (0/2 tries). Called with (*('evanston, Illinois',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Library/Frameworks/Py

RateLimiter caught an error, retrying (0/2 tries). Called with (*('glenview, Illinois',), **{}).
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Library/Frameworks/Py

KeyboardInterrupt: 

In [84]:
total_df.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,incident_city,unit,incident_end_date,weekday,season_356,season_fall,season_spring,season_summer,season_winter,incident_length
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,22.0,0,0,0,1,0,0,0,0,0,...,unknown,unknown,1900-01-01 00:00:00,0.0,0,0,1,0,0,0
272161011760,29.0,0,1,0,1,0,0,0,0,0,...,unknown,unknown,1900-01-01 00:00:00,1.0,0,0,0,1,0,0
864286527653,29.0,0,0,0,1,0,0,0,0,0,...,unknown,unknown,1900-01-01 00:00:00,1.0,0,0,1,0,0,0
882206007016,34.0,0,0,0,1,0,0,0,0,0,...,chicago,unknown,1900-01-01 00:00:00,1.0,0,1,0,0,0,0
882242005211,41.0,0,0,0,1,0,0,0,0,0,...,chicago,unknown,1900-01-01 00:00:00,0.0,0,0,0,1,0,0


In [85]:
total_df.shape

(289094, 4837)

# Final Data Prep: Scaling, Train/Test Split, Downsampling

In [86]:
''' 
Create final_df from copy of total_df
Removing "unsquishable" categorical columns from total_df that we added back in for engineering
'''

final_df = total_df.copy()
to_remove = ['event_date', 'incident_begin_date', 'arrest_date', 'arraignment_date', 'incident_city', 'unit']
final_df = final_df.drop(columns=to_remove)

In [87]:
final_df.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,charge_offense_title_wrker comp/fraudulent claim,MHI,incident_end_date,weekday,season_356,season_fall,season_spring,season_summer,season_winter,incident_length
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,22.0,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,1,0,0,0
272161011760,29.0,0,1,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,0,1,0,0
864286527653,29.0,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,1,0,0,0
882206007016,34.0,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,1,0,0,0,0
882242005211,41.0,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,0,1,0,0


In [ ]:
final_df.shape

In [92]:
'''
SCALING / NORMALIZATION

kcm code expanded by akr

turns total_df into final_df, final_df becomes our working copy

akr notes/questions:
we should add all other numeric columns that we create
is it ok that we're normalizing before downsampling?
did i expand this code right? the "since only one column" thing makes me think maybe not
kcm: i think so, thats just what you do if its just one series'''

from sklearn.preprocessing import StandardScaler

age = final_df['age_at_incident'].values.reshape(-1, 1) #reshaping since only one column
scaler = StandardScaler().fit(age)
final_df['age_at_incident'] = scaler.transform(age) 
#print(final_df['age_at_incident'].mean()) #~ 0

In [93]:
final_df.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,charge_offense_title_wrker comp/fraudulent claim,MHI,incident_end_date,weekday,season_356,season_fall,season_spring,season_summer,season_winter,incident_length
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,-0.963778,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,1,0,0,-0.062916
272161011760,-0.397071,0,1,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,0,1,0,-0.062916
864286527653,-0.397071,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,1,0,0,-0.062916
882206007016,0.007720,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,1,0,0,0,-0.062916
882242005211,0.574427,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,0,1,0,-0.062916


In [95]:



#THIS LINE NOOT WORKING 

# comment this out

# days_since_received = final_df['days_since_received'].values.reshape(-1, 1) #reshaping since only one column
# scaler = StandardScaler().fit(days_since_received)
# final_df['days_since_received'] = scaler.transform(days_since_received) 
# print(final_df['days_since_received'].mean()) #~ 0


In [96]:

charge_count = final_df['charge_count'].values.reshape(-1, 1) #reshaping since only one column
scaler = StandardScaler().fit(charge_count)
final_df['charge_count'] = scaler.transform(charge_count) 

final_df.shape

(289094, 4830)

In [97]:


incident_length = final_df['incident_length'].values.reshape(-1, 1) #reshaping since only one column
scaler = StandardScaler().fit(incident_length)
final_df['incident_length'] = scaler.transform(incident_length)
#print(final_df['incident_length'].mean()) #~ 0

final_df.shape

(289094, 4830)

In [100]:
final_df.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,charge_offense_title_wrker comp/fraudulent claim,MHI,incident_end_date,weekday,season_356,season_fall,season_spring,season_summer,season_winter,incident_length
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,-0.963778,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,1,0,0,-0.062916
272161011760,-0.397071,0,1,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,0,1,0,-0.062916
864286527653,-0.397071,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,1,0,0,-0.062916
882206007016,0.007720,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,1,0,0,0,-0.062916
882242005211,0.574427,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,0,1,0,-0.062916


In [106]:
final_df['chapter_15'].unique()

array([-8.63698874e-03,  2.58291531e+02,  4.30413910e+01,  8.60914190e+01,
        3.44391587e+02])

In [108]:
final_df['chapter_15'].head(20)

case_participant_id
260122253823   -0.008637
272161011760   -0.008637
864286527653   -0.008637
882206007016   -0.008637
882242005211   -0.008637
882325141108   -0.008637
882415546999   -0.008637
883042173560   -0.008637
883053547582   -0.008637
883054133872   -0.008637
883054602904   -0.008637
883055071936   -0.008637
883055189194   -0.008637
883055423710   -0.008637
883056361774   -0.008637
883056479032   -0.008637
883056830805   -0.008637
883057299837   -0.008637
883057417095   -0.008637
883057768869   -0.008637
Name: chapter_15, dtype: float64

In [110]:
# NOT RUNNING THIS FOR NOW

# # need to add all dummy features created by the to_sum list. will be sparse
# # ie, any key for which the value is 'sum' in agg_dict_long
# to_sum = ['chapter', 'act', 'section', 'class', 'aoic', 'charge_offense_title']

# for variable in to_sum:

#   # list of columns starting with variable
#   filter_col = [col for col in final_df if col.startswith(variable)]

#   for column in filter_col:
#     var = final_df[column].values.reshape(-1, 1) 
#     scaler = StandardScaler().fit(var)
#     final_df[column] = scaler.transform(var) 

# # need to add incident length. will be sparse
# # kcm will handle by tues 12/3 meeting

final_df.shape

(289094, 4830)

In [ ]:
# AMBER - GOING OK UNTIL HERE!!! 

In [111]:
final_df.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,charge_offense_title_wrker comp/fraudulent claim,MHI,incident_end_date,weekday,season_356,season_fall,season_spring,season_summer,season_winter,incident_length
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,-0.963778,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,1,0,0,-0.062916
272161011760,-0.397071,0,1,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,0,1,0,-0.062916
864286527653,-0.397071,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,1,0,0,-0.062916
882206007016,0.007720,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,1,0,0,0,-0.062916
882242005211,0.574427,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,0,1,0,-0.062916


In [123]:
# # Save final_df to drive
import pickle 
# from google.colab import drive
# drive.mount('drive')

# final_df.to_csv('final_df.csv')
# !cp final_df.csv drive/My\ Drive/
final_df.to_csv('final_df_csv.csv')
# with open('final_df_1.pkl', 'wb') as f:
#     final_df.to_pickle(f)



In [124]:
final_df.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,charge_offense_title_wrker comp/fraudulent claim,MHI,incident_end_date,weekday,season_356,season_fall,season_spring,season_summer,season_winter,incident_length
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,-0.963778,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,1,0,0,-0.062916
272161011760,-0.397071,0,1,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,0,1,0,-0.062916
864286527653,-0.397071,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,1,0,0,-0.062916
882206007016,0.007720,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,1,0,0,0,-0.062916
882242005211,0.574427,0,0,0,1,0,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,0,1,0,-0.062916


In [126]:
finsl_saved = pd.read_csv('final_df_csv.csv')

In [128]:
finsl_saved.head()

,case_participant_id,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,...,charge_offense_title_wrker comp/fraudulent claim,MHI,incident_end_date,weekday,season_356,season_fall,season_spring,season_summer,season_winter,incident_length
0,260122253823,-0.963778,0,0,0,1,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,1,0,-0.601008,-0.062916
1,272161011760,-0.397071,0,1,0,1,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,0,1,-0.601008,-0.062916
2,864286527653,-0.397071,0,0,0,1,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,0,1,0,-0.601008,-0.062916
3,882206007016,0.007720,0,0,0,1,0,0,0,0,...,0,0,1900-01-01 00:00:00,1.0,0,1,0,0,-0.601008,-0.062916
4,882242005211,0.574427,0,0,0,1,0,0,0,0,...,0,0,1900-01-01 00:00:00,0.0,0,0,0,1,-0.601008,-0.062916


In [129]:
finsl_saved.shape

(289094, 4831)

In [130]:
final_df['received_date'] = total_df['received_date']

In [131]:
final_df.head()

,age_at_incident,age_over_100,age_unknown,gender_female,gender_male,gender_unknown,race_albino,race_american indian,race_asian,race_biracial,...,MHI,incident_end_date,weekday,season_356,season_fall,season_spring,season_summer,season_winter,incident_length,received_date
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,-0.963778,0,0,0,1,0,0,0,0,0,...,0,1900-01-01 00:00:00,0.0,0,0,1,0,0,-0.062916,2011-05-24 00:00:00
272161011760,-0.397071,0,1,0,1,0,0,0,0,0,...,0,1900-01-01 00:00:00,1.0,0,0,0,1,0,-0.062916,2012-01-27 00:00:00
864286527653,-0.397071,0,0,0,1,0,0,0,0,0,...,0,1900-01-01 00:00:00,1.0,0,0,1,0,0,-0.062916,2011-01-31 00:00:00
882206007016,0.007720,0,0,0,1,0,0,0,0,0,...,0,1900-01-01 00:00:00,1.0,0,1,0,0,0,-0.062916,2011-01-31 00:00:00
882242005211,0.574427,0,0,0,1,0,0,0,0,0,...,0,1900-01-01 00:00:00,0.0,0,0,0,1,0,-0.062916,2011-07-17 00:00:00


In [132]:
final_df.to_csv('final_df_120419.csv')

In [119]:
'''
Split Data into Train / Validate / Test Sets

current method: 'timeline'
ensures that we only ever predict forward in time

should be executed on scaled df, ie final_df
'''


def get_train_val_test (df, method, train_size, test_size, val_size): 
  #takes in df (assumes 'MHI' is target variable)
  #methods are strings, defined below
  #takes in train, test, and val sizes (fractions of 1)
  #returns xtrain, xval, xtest, ytrain, yval, ytest

  #current method: 'timeline'

  #list of methods that are finished and ready to be used
  possible_methods = ['simple', 'timeline']

  #check that inputted method is in possible_methods
  if method not in possible_methods:
      print('error. possible methods: ', possible_methods)
      return 
  
  if method=='simple':
    #static validation set, random sampling via shuffle
    #uses train_size, test_size, val_size

    #check that specificed sizes sum to 1
    if train_size + test_size+val_size != 1:
      print('error: train_size + test_size + val_size must sum to 1')
      return 
      
    #set x and y
    x = df.drop('MHI', axis=1)
    y = df['MHI']

    #first split test from the other 2
    xtrainval, xtest, ytrainval, ytest = train_test_split(x, y, test_size=test_size, shuffle=True, random_state=2019)
    
    #second split train and val
    xtrain, xval, ytrain, yval = train_test_split(xtrainval, ytrainval, test_size=val_size/(val_size+train_size), shuffle=True, random_state=2019)
    return xtrain, xval, xtest, ytrain, yval, ytest


  if method=='timeline':
    #static validation set, divisions based on timeline
    #should reduce leakage
    #danger of bias bc training set will have higher proportion of "finalized" rows

    #check that specificed sizes sum to 1
    if train_size + test_size + val_size != 1:
      print('error: train_size + test_size + val_size must sum to 1')
      return 

    #sort ascending by received date
    sorted_df = df.sort_values(by=['received_date'])

    #set x and y
    x = sorted_df.drop('MHI', axis=1)
    y = sorted_df['MHI']

    total_size = len(x)

    #first train_size % indices in train
    xtrain = x.iloc[:int(round(train_size*total_size))]
    ytrain = y.iloc[:int(round(train_size*total_size))]

    #next val_size % indices in val
    xval = x.iloc[int(round(train_size*total_size)):int(round((train_size+val_size)*total_size))]
    yval = y.iloc[int(round(train_size*total_size)):int(round((train_size+val_size)*total_size))]

    #final test_size % indices in test
    xtest = x.iloc[int(round((train_size+val_size)*total_size)):-1]
    ytest = y.iloc[int(round((train_size+val_size)*total_size)):-1]

    return xtrain, xval, xtest, ytrain, yval, ytest

  if method=='walk':
    #walk forward
    return

In [120]:
# execute the split on final_df
X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test (final_df, 'timeline', 0.7, 0.15, 0.15)

KeyError: 'received_date'

In [0]:
'''
Downsampling the training set

at function tweaked by akr
'''

from random import sample 

def downsample(df, pct_MHI1): 
  # takes in percentage from 1 to 50
  # samples all MHI==1 cases
  # samples from MHI==0 cases such that downsampled_df has pct_MHI1 % positive cases

  # split into MHI 1 and MHI 0 
  MHI1 = df[df['MHI'] == 1]
  MHI0 = df[df['MHI'] == 0]

  # count number of MHIs
  count_MHI1 = len(MHI1)

  # compute number of negative cases to sample
  num_MHI0 = count_MHI1 * int(round((100-pct_MHI1)/pct_MHI1))

  # sample from negative cases
  MHI0_sample = MHI0.sample(n=num_MHI0, random_state=42)

  # append sampled negative cases to all positive cases
  downsampled_df = MHI1.append(MHI0_sample)

  return downsampled_df

training_df = X_train.join(y_train)

# passing 50 to downsample function means training set will have 50% positive cases
# we can vary this downsampling call between 1% and 50%
downsampled_training_df = downsample(training_df, 50)

X_train_down = downsampled_training_df.drop('MHI', axis=1)
y_train_down = downsampled_training_df['MHI']

# Random Forest Baseline Model

In [0]:
'''
KCM Random Forest Baseline
'''

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score

non_baseline = ['402', 'weekday', 'season', 'incident_length', 'age_over_100', 'age_unknown']
# 'latitude', 'longitude'

base_X_train_down = X_train_down.drop(columns=non_baseline)
base_X_val = X_val.drop(columns=non_baseline)
base_X_test.drop(columns=non_baseline)

clf = RandomForestClassifier(random_state=0).fit(base_X_train_down,y_train)
y_pred_proba = clf.predict_proba(base_X_val)
y_pred = clf.predict(base_X_val)
print("Accuracy:", metrics.accuracy_score(y_val, y_pred))
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:,1])
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)

In [0]:
'''
Feature Importance Plot for Random Forest
'''

import matplotlib.pyplot as plt
ftimp = [x for x in zip(X.columns, clf.feature_importances_)] #list of tuples
ftimp.sort(key = lambda x: x[1]) #sorting based on feature importance
ftimp = [x for x in ftimp if x[1] > 0.01] #theres a ton above 0
plt.figure(figsize=(15,15))
plt.bar(*zip(*ftimp)) #unzipping ftimp
plt.xticks(rotation='vertical')
plt.show()

# Data Analysis

In [0]:
''' 
FEATURE ENGINEERING FOR DATA ANAYLSIS:
create days since received column

subtracting from data upload date of July 26,2019

chose received because it's the only "start" date without missing values
'''

upload_date = pd.to_datetime(pd.datetime.strptime('7/26/2019', '%m/%d/%Y'))
received_date_dt = pd.to_datetime(total_df.received_date)
total_df['days_since_received'] = (upload_date - received_date_dt).astype(str)
total_df['days_since_received'] = total_df['days_since_received'].map(lambda x: x.partition(' ')[0])
total_df['days_since_received'] = total_df['days_since_received'].astype(int)

In [0]:
'''
Data Analysis:
Distribution of MHI over Time
'''

import matplotlib.pyplot as plt
%matplotlib inline


def get_portion_true (decile):
  #returns the portion of MHI==1 out of the total decile
  num_true = len(decile[decile['MHI']==1])
  return (num_true/len(decile))

def plot_deciles(df, col):

  #get the total number of rows in the df
  n = len(df)

  #sort df by column of interest
  sorted_df = df.sort_values(by=[col])

  #create deciles, ie bins of the column of interest
  decile_1 = sorted_df.iloc[0:round(n/10)]
  decile_2 = sorted_df.iloc[round(n/10):round(2*n/10)]
  decile_3 = sorted_df.iloc[round(2*n/10):round(3*n/10)]
  decile_4 = sorted_df.iloc[round(3*n/10):round(4*n/10)]
  decile_5 = sorted_df.iloc[round(4*n/10):round(5*n/10)]
  decile_6 = sorted_df.iloc[round(5*n/10):round(6*n/10)]
  decile_7 = sorted_df.iloc[round(6*n/10):round(7*n/10)]
  decile_8 = sorted_df.iloc[round(7*n/10):round(8*n/10)]
  decile_9 = sorted_df.iloc[round(8*n/10):round(9*n/10)]
  decile_10 = sorted_df.iloc[round(9*n/10):n]

  #list of deciles
  deciles = [decile_1, decile_2, decile_3, decile_4, decile_5, 
           decile_6, decile_7, decile_8, decile_9, decile_10]

  #create list of portions
  portions = [get_portion_true(x) for x in deciles]

  #create string labels of earliest arrest date in decile
  labels = [str(x.iloc[0][col]) for x in deciles]

  #plot
  fig, ax = plt.subplots()
  ax.bar(labels, portions, width=.55)
  ax.set_xticklabels(labels, rotation = 90)
  ax.set_ylabel('Portion of records with MHI==1')
  plt.title('MHI by {}'.format(col))
  plt.show()

  return

time_features_to_plot = ['event_date', 'received_date', 'arrest_date', 'incident_begin_date']

for column in time_features_to_plot:
  plot_deciles(total_df, column)


In [0]:
'''
CENSORING PROBLEM ANALYSIS
Looking at the degree to which the "rolling in" effects our base rates
Should ultimately select cut-off heuristic (per Brian)

Need to create scatter plot
X axis: age of case
Y axis: MHI base rate
how to do this, though? binning?

Note: Need to do research on censoring problem
Brian told me to research censoring in survival analysis
'''


In [0]:
''' 
Data Analysis:
Looking at MHI Base Rates
Across Train/Validation/Test
'''
  
training_df = X_train.join(y_train)
val_df = X_val.join(y_val)
train_and_val_df = pd.concat([training_df, val_df], verify_integrity=True)
test_df = X_val.join(y_val)


print ("Training base rate: ", get_portion_true(training_df))
print ("Validation base rate: ", get_portion_true(val_df))
print ("Training+Validation base rate: ", get_portion_true(train_and_val_df))
print ("Test base rate: ", get_portion_true(test_df))